<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/R/colab/loading-networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Networks
## Kozo Nishida, Kristina Hanspers and Alex Pico


*The R markdown is available. [download the Rmd from GitHub](https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/R/notebooks/loading-networks.Rmd).*

<hr />

In Cytoscape, network data can be loaded from a variety of sources, and in several different formats. Where you get your network data depends on your biological question and analysis plan. This tutorial outlines how to load network data from several popular sources and formats.

1. Public databases
  - **NDEx**
  - **PSICQUIC**
  - **STRING/STITCH**
  - **WikiPathways**
2. Local and remote files
3. Cytoscape apps (Biopax, KEGG and other formats)

<hr />

# Installation

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

if(!"RCy3" %in% installed.packages())
  BiocManager::install("RCy3")

library(RCy3)

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)

In [ ]:
# Connect to local Cytoscape
browserClientJs <- getBrowserClientJs()
IRdisplay::display_javascript(browserClientJs)

# Getting started
First, launch Cytoscape and keep it running whenever using RCy3. Confirm that you have everything installed and running:

In [ ]:
cytoscapePing()
cytoscapeVersionInfo()

# Prerequisites

The exercises require you to have certain Cytoscape apps and R packages installed.

In [ ]:
installApp('stringApp')
installApp('WikiPathways')

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("ndexr")
BiocManager::install("rWikiPathways")

install.packages("httr")

# Networks from Public Data

Cytoscape includes a **Network Search** tool for easy import of public network data.
In addition to core apps that are included with your Cytoscape installation (NDEx and PSICQUIC), the resources listed here will depend on which apps you have installed.

Find out which apps you have installed:

In [ ]:
getInstalledApps()

# NDEx

The [NDEx Project](http://www.ndexbio.org/) provides an open-source framework where scientists and organizations can share, store, manipulate, and publish biological network knowledge.

Search NDEx for "TP53 AND BARD1":

In [ ]:
library(ndexr)
ndexcon <- ndex_connect()
networks <- ndex_find_networks(ndexcon, "TP53 AND BARD1")
print(networks[,c("name","externalId","nodeCount","edgeCount")])

We can work with this list. Let’s use the first hit. **Note: you are going to get different hits as this database changes over time, so proceed with any hit you like.**

In [ ]:
networkId = networks$externalId[1]
network = ndex_get_network(ndexcon, networkId)
print(network)

Import the network into Cytoscape:

In [ ]:
importNetworkFromNDEx(networkId)

For more detailed information about working with NDEx networks in Cytoscape, see the [Cytoscape-and-NDEx](https://cytoscape.org/cytoscape-automation/for-scripters/R/notebooks/Cytoscape-and-NDEx.nb.html) protocol.

# STRING/STITCH

STRING is a database of known and predicted protein-protein interactions, and STITCH stored known and predicted interactions between chemicals and proteins. Data types include:

- Genomic Context Predictions
- High-throughput Lab Experiments
- (Conserved) Co-Expression
- Automated Textmining
- Previous Knowledge in Databases

Search STRING with the disease keyword "ovarian cancer". The resulting network will load automatically.

In [ ]:
string.cmd = 'string disease query disease="ovarian cancer"'
commandsRun(string.cmd)

STRING networks load with a STRING-specific style, which includes 3D protein structure diagrams.

Export the image as a png. This will save the png to your current directory.

In [ ]:
exportImage('ovarian_cancer', 'PNG')

STRING networks also include data as node/interaction attributes, that can be used to create a **Style**.
Let's save the attributes as a dataframe and take a look at the first few rows:

In [ ]:
df <- getTableColumns()
head(df)

The STRING app includes options to change interaction confidence level, expand the network etc.

Before changing interaction confidence level, let's find the number of interactions in the network:

In [ ]:
getEdgeCount()

Let's increase the confidence level to 0.9, from the default 0.4:

In [ ]:
string.cmd = 'string change confidence confidence=0.9 network=CURRENT'
commandsRun(string.cmd)

Now let's get an edge count after changing interaction confidence level:

In [ ]:
getEdgeCount()

Again, we can export a figure:

In [ ]:
exportImage('before_expand', 'PNG')

For more detailed information about working with STRING networks in Cytoscape, see the [stringApp](https://cytoscape.org/cytoscape-automation/for-scripters/R/notebooks/stringApp.nb.html) protocol.

# WikiPathways

[WikiPathways](https://www.wikipathways.org/) is a collaborative wiki platform with manually pathway models. It currently covers over 2,800 pathways in 29 species-specific collections.

First, load the WikiPathways R library:

In [ ]:
library(rWikiPathways)

Search WikiPathways for 'statin':

In [ ]:
statin.pathways <- findPathwaysByText('statin')

WikiPathways search results are displayed as a list of all available pathway models matching the search term.

In [ ]:
head(statin.pathways)

We can restrict the results to **Homo sapiens**:

In [ ]:
human.filter <- lapply(statin.pathways, function(x) x$species == "Homo sapiens")
human.statin.pathways <- statin.pathways[unlist(human.filter)]

Loading the pathways will by default preserve the original pathway layout:

In [ ]:
lapply(human.statin.pathways, function(x) commandsRun(paste('wikipathways import-as-pathway id=', x$id, sep = '')))

The pathway can also be opened as a network:

In [ ]:
lapply(human.statin.pathways, function(x) commandsRun(paste('wikipathways import-as-network id=', x$id, sep = '')))

WikiPathways model imported as **pathway** and **network**.
![](https://cytoscape.org/cytoscape-tutorials/protocols/loading-networks/WP-pathway.png){width=70%} ![](https://cytoscape.org/cytoscape-tutorials/protocols/loading-networks/WP-network.png)

For more detailed information about working with WikiPathways in Cytoscape, see the [rWikiPathways and rCy3](https://cytoscape.org/cytoscape-automation/for-scripters/R/notebooks/rWikiPathways-and-RCy3.nb.html) protocol.

# Local and Remote Files

Cytoscape can load locally and remotely stored network data files in a variety of file formats:

- **SIF**: Simple interaction format
- **NNF**: Nested network format
- **GML** and **XGMML** formats
- **CYS**: Cytoscape session file
- Delimited text and Excel format

## Loading SIF files

**SIF** is a simple interaction format consisting of three columns of data: source, interaction and target. To learn more about the SIF format, see the [Cytoscape manual](http://manual.cytoscape.org/en/stable/Supported_Network_File_Formats.html#sif-format).

Download [*galFiltered.sif*](https://cytoscape.github.io/cytoscape-tutorials/protocols/data/galFiltered.sif) and load the network via:

In [ ]:
library(httr)
sif_url = "https://cytoscape.github.io/cytoscape-tutorials/protocols/data/galFiltered.sif"
GET(sif_url, write_disk(tf <- tempfile(fileext = ".sif")))
importNetworkFromFile(tf)

To see the whole network, use fitContent:

In [ ]:
fitContent()

## Loading XGMML files

**XGMML** is an XML format and can includes node and edge attributes as well as visual style properties. To learn more about the XGMML format, see the [Cytoscape manual](http://manual.cytoscape.org/en/stable/Supported_Network_File_Formats.html#xgmml-format).

Download https://raw.githubusercontent.com/cytoscape/cytoscape-tutorials/gh-pages/protocols/data/BasicDataVizDemo.xgmml and load the network via

In [ ]:
xgmml_url = "https://raw.githubusercontent.com/cytoscape/cytoscape-tutorials/gh-pages/protocols/data/BasicDataVizDemo.xgmml"
GET(xgmml_url, write_disk(tf <- tempfile(fileext = ".xgmml")))
importNetworkFromFile(tf)

This is the same network as in the previous step, but with node fill color corresponding to expression values. ![](https://cytoscape.org/cytoscape-tutorials/protocols/loading-networks/galFiltered2Loaded.png)